In [ ]:
#import torch and cuda

import torch
import torch.nn.functional as F

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

In [ ]:
#Configure device: Set the device configuration to GPU using the torch.device class in PyTorch:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
##test fine tune

#load
from datasets import load_dataset
imdb = load_dataset("imdb")


In [ ]:
from datasets.utils.logging import get_logger
import os

# Get the cache directory
cache_dir = os.getenv('HF_DATASETS_CACHE', os.path.expanduser("~/.cache/huggingface/datasets"))
print("Hugging Face datasets cache directory:", cache_dir)

In [ ]:
#preprocess

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
import os
from transformers import AutoTokenizer

# Retrieve the cache directory used by Hugging Face
cache_dir = os.getenv('HF_HOME', os.path.expanduser("~/.cache/huggingface/transformers"))
print("Cache directory:", cache_dir)


In [ ]:
def preprocess_data(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = imdb.map(preprocess_data, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
from transformers import DataCollatorWithPadding

# Adjust training arguments for GTX 1650
training_args = TrainingArguments(
    output_dir="my_awesome_model",          # Directory to save the model
    learning_rate=2e-5,                     # Fine-tuning learning rate
    per_device_train_batch_size=8,          # Reduce batch size for GPU memory
    per_device_eval_batch_size=8,           # Reduce evaluation batch size
    num_train_epochs=1,                     # Number of epochs (adjust if needed)
    weight_decay=0.01,                      # Weight decay
    eval_strategy="epoch",                  # Evaluate at the end of each epoch
    save_strategy="epoch",                  # Save checkpoint at the end of each epoch
    load_best_model_at_end=True,            # Load the best model
    push_to_hub=False,                      # Disable pushing to the Hugging Face Hub
    fp16=True,                              # Enable mixed precision training
    gradient_accumulation_steps=2,          # Accumulate gradients to simulate larger batch size
    logging_dir="./logs",                   # Directory for logs
    logging_steps=50,                       # Adjust logging frequency
)

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Use a smaller dataset for quick experimentation
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(4000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# Define trainer with optimizations
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,     # Use smaller training dataset
    eval_dataset=small_eval_dataset,       # Use smaller evaluation dataset
    tokenizer=tokenizer,                   # Required for Trainer
    data_collator=data_collator,           # Dynamic padding
    compute_metrics=compute_metrics,       # Custom metrics (accuracy, etc.)
)

# Start training
trainer.train()


In [ ]:
eval_results = trainer.evaluate()

In [ ]:
trainer.save_model("my_awesome_model")

In [ ]:
text = "I love watching rent a girlfriend because the character especially Chizuru Ichinose is beautiful, and elegant"

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="my_awesome_model")
classifier(text)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_model")
inputs = tokenizer(text, return_tensors="pt")



In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("my_awesome_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]